# 1단계: HardlyHumans 모델로 감정 예측

**목적:** 이 노트북은 `HardlyHumans/Facial-expression-detection` 모델을 로드하고, `Data/EST_data` 디렉토리의 이미지(train, validation, test 세트)를 처리하여 예측을 생성합니다. 통합된 결과를 단일 CSV 파일로 저장합니다.

**이전 버전과의 주요 변경 사항:**
- `map_label` 함수 (fear -> panic)가 제거되었습니다.
- 단일 함수가 `img_train`, `img_val`, `img_test` 디렉토리를 처리합니다.
- `dataset` 열이 포함된 `02_prediction_results_all.csv`라는 단일 CSV 파일을 출력합니다.

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline, ViTImageProcessor
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
import est_utils as utils # est_utils 모듈 임포트

# 사전 훈련된 모델 및 프로세서 로드
model_name = "HardlyHumans/Facial-expression-detection"
pipe = pipeline("image-classification", model=model_name, device=0) # GPU 사용 가능 시 GPU 사용

# processor = AutoImageProcessor.from_pretrained(model_name)
processor = ViTImageProcessor.from_pretrained(model_name, use_fast=True)
model = AutoModelForImageClassification.from_pretrained(model_name)

labels = model.config.id2label  # 모델 클래스 라벨(8가지)
print(f"모델 라벨: {labels}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/workspace/miniconda3/envs/mlflow_env/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Device set to use cuda:0


모델 라벨: {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise'}


### 이미지 파일 탐색 함수

In [2]:
def get_image_files(base_dir, img_splits, classes):
    """
    지정된 데이터셋 유형 폴더(img_train, img_val, img_test)의 모든 이미지 파일을 찾습니다.
    이미지 경로, 실제 라벨, 데이터셋 유형이 포함된 딕셔너리 리스트를 반환합니다.
    """
    all_files = []
    for split in img_splits:
        data_dir = os.path.join(base_dir, split)
        if not os.path.isdir(data_dir):
            print(f"경고: 디렉토리를 찾을 수 없어 건너뜁니다: {data_dir}")
            continue
            
        for emotion_folder in classes:
            emotion_dir = os.path.join(data_dir, emotion_folder)
            if os.path.isdir(emotion_dir):
                for filename in os.listdir(emotion_dir):
                    if filename.lower().endswith((".jpg", ".jpeg")):
                        all_files.append({
                            'image_path': os.path.join(emotion_dir, filename),
                            'true_label': emotion_folder,
                            'dataset': split.replace('img_', '')
                        })
    return all_files

### 이미지 처리 및 예측 생성

In [3]:
image_files_to_process = get_image_files(utils.BASE_DIR, utils.IMG_SPLITS, utils.CLASSES)

if not image_files_to_process:
    print("이미지 파일을 찾을 수 없습니다. 디렉토리 구조와 경로를 확인하세요.")
else:
    print(f"처리할 이미지 {len(image_files_to_process)}개를 찾았습니다.")
    results = []
    for item in tqdm(image_files_to_process, desc="이미지 처리 중"):
        try:
            img = Image.open(item['image_path']).convert('RGB')
            prediction = pipe(img)
            predicted_label = prediction[0]['label']
            results.append({
                'image_path': item['image_path'],
                'true_label': item['true_label'],
                'predicted_label': predicted_label,
                'dataset': item['dataset']
            })
        except Exception as e:
            print(f"{item['image_path']} 처리 중 오류 발생: {e}")
            
    # 데이터프레임을 생성하고 단일 CSV 파일로 저장
    results_df = pd.DataFrame(results)
    output_path = '02_prediction_results_all.csv'
    results_df.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n총 {len(results_df)}개의 이미지를 성공적으로 처리했습니다.")
    print(f"결과가 {output_path}에 저장되었습니다.")

처리할 이미지 8396개를 찾았습니다.


이미지 처리 중: 100%|██████████| 8396/8396 [15:24<00:00,  9.08it/s]


총 8396개의 이미지를 성공적으로 처리했습니다.
결과가 03_prediction_results_all.csv에 저장되었습니다.


### 결과 샘플 표시

In [4]:
if 'results_df' in locals():
    print("생성된 데이터 샘플:")
    display(results_df.head())

생성된 데이터 샘플:


,image_path,true_label,predicted_label,dataset
0,/workspace/AI/FER/Data/EST_data/img_train/ange...,anger,surprise,train
1,/workspace/AI/FER/Data/EST_data/img_train/ange...,anger,neutral,train
2,/workspace/AI/FER/Data/EST_data/img_train/ange...,anger,neutral,train
3,/workspace/AI/FER/Data/EST_data/img_train/ange...,anger,neutral,train
4,/workspace/AI/FER/Data/EST_data/img_train/ange...,anger,neutral,train
